In [1]:
%load_ext autoreload


In [2]:
%autoreload 2 

import account
name  = account.account_get("kogas", "manager_name")
phone = account.account_get("kogas", "manager_phone")
email = account.account_get("kogas", "manager_email")

from org.kogas.kogas import Kogas
kogas = Kogas(name, phone, email)

In [6]:
%autoreload 2
kogas.login()


Error: Failed to find 로그인 button


Error element not found


False

In [38]:


kogas.register("2022112402")

UnexpectedAlertPresentException: Alert Text: 가스공사에 입찰등록을 제출하시겠습니까?

(동일 사항에 대하여 동일 PC에서는 1통의 입찰서만을 제출할 수 있습니다.)
Message: unexpected alert open: {Alert text : 가스공사에 입찰등록을 제출하시겠습니까?

(동일 사항에 대하여 동일 PC에서는 1통의 입찰서만을 제출할 수 있습니다.)}
  (Session info: MicrosoftEdge=107.0.1418.56)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF78ACE8532+9986]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78AC85D62+1445202]
	Ordinal0 [0x00007FF78A86FC8C+654476]
	Ordinal0 [0x00007FF78A8ED828+1169448]
	Ordinal0 [0x00007FF78A8D39B3+1063347]
	Ordinal0 [0x00007FF78A8A506A+872554]
	Ordinal0 [0x00007FF78A8A402E+868398]
	Ordinal0 [0x00007FF78A8A570F+874255]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78AB46108+135416]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78AB302CF+45759]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF78AB3374C+59196]
	Ordinal0 [0x00007FF78A

In [70]:
%autoreload 2 
kogas.participate("2022111702001-00", "24056199")


True